<a href="https://colab.research.google.com/github/saishshinde15/Translation_NLPs/blob/main/Englsih_Hindi_Machine_translation_encode_decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [181]:
dataset=pd.read_csv('data 2.csv')

In [182]:
dataset

,english,spanish
0,Go.,Ve.
1,Go.,Vete.
2,Go.,Vaya.
3,Go.,Váyase.
4,Hi.,Hola.
...,...,...
118959,There are four main causes of alcohol-related ...,Hay cuatro causas principales de muertes relac...
118960,There are mothers and fathers who will lie awa...,Hay madres y padres que se quedan despiertos d...
118961,A carbon footprint is the amount of carbon dio...,Una huella de carbono es la cantidad de contam...
118962,Since there are usually multiple websites on a...,Como suele haber varias páginas web sobre cual...


In [183]:
dataset.isnull().sum()

english    0
spanish    0
dtype: int64

In [184]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118964 entries, 0 to 118963
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   english  118964 non-null  object
 1   spanish  118964 non-null  object
dtypes: object(2)
memory usage: 1.8+ MB


In [185]:
dataset=dataset.head(2000)

In [186]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   english  2000 non-null   object
 1   spanish  2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


In [187]:
X=dataset['english']

In [188]:
X

0                Go.
1                Go.
2                Go.
3                Go.
4                Hi.
            ...     
1995    It happened.
1996    It happened.
1997    It is on me.
1998    It is there.
1999    It may rain.
Name: english, Length: 2000, dtype: object

In [189]:
y=dataset['spanish']

In [190]:
y

0                     Ve.
1                   Vete.
2                   Vaya.
3                 Váyase.
4                   Hola.
              ...        
1995                Pasó.
1996         Eso sucedió.
1997           Yo invito.
1998            Está ahí.
1999    Puede que llueva.
Name: spanish, Length: 2000, dtype: object

In [191]:
tokenizer_eng=Tokenizer(num_words=5000,oov_token="<OOV>")#ovv_token="<OOV>"
tokenizer_eng.fit_on_texts(X)
X=tokenizer_eng.texts_to_sequences(X)

In [192]:
tokenizer_spa=Tokenizer(num_words=5000,oov_token="<OOV>")
tokenizer_spa.fit_on_texts(y)
y=tokenizer_spa.texts_to_sequences(y)

In [193]:
X=pad_sequences(X)

In [194]:
x_dim=X.shape[1]
x_dim

4

In [195]:
y=pad_sequences(y,maxlen=x_dim)

In [196]:
V=len(tokenizer.word_index)
V

1528

In [197]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [198]:
len(X_train)

1600

In [199]:
len(y_train)

1600

In [200]:
len(y_test)

400

In [201]:
# Define the encoder
encoder_inputs = tf.keras.Input(shape=(x_dim,))
encoder_embedding = tf.keras.layers.Embedding(V+1, 64)(encoder_inputs)
encoder_lstm = tf.keras.layers.LSTM(64, return_sequences=True)(encoder_embedding)
encoder_outputs, state_h, state_c = tf.keras.layers.LSTM(64, return_state=True)(encoder_lstm)
encoder_states = [state_h, state_c]

# Define the decoder
decoder_inputs = tf.keras.Input(shape=(x_dim,))
decoder_embedding = tf.keras.layers.Embedding(V+1, 64)(decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM(64, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = tf.keras.layers.Dense(V, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [202]:
# Define the model
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=decoder_outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit([X_train, y_train], y_train, epochs=10, verbose=1)



Epoch 1/10
50/50 [==============================] - 15s 111ms/step - loss: 6.2704 - accuracy: 0.4156
Epoch 2/10
50/50 [==============================] - 5s 93ms/step - loss: 4.2353 - accuracy: 0.4250
Epoch 3/10
50/50 [==============================] - 3s 61ms/step - loss: 3.9264 - accuracy: 0.4250
Epoch 4/10
50/50 [==============================] - 3s 56ms/step - loss: 3.6917 - accuracy: 0.4250
Epoch 5/10
50/50 [==============================] - 5s 104ms/step - loss: 3.5697 - accuracy: 0.4250
Epoch 6/10
50/50 [==============================] - 3s 58ms/step - loss: 3.4855 - accuracy: 0.4250
Epoch 7/10
50/50 [==============================] - 5s 97ms/step - loss: 3.4139 - accuracy: 0.4250
Epoch 8/10
50/50 [==============================] - 7s 147ms/step - loss: 3.3479 - accuracy: 0.4250
Epoch 9/10
50/50 [==============================] - 7s 134ms/step - loss: 3.2883 - accuracy: 0.4289
Epoch 10/10
50/50 [==============================] - 8s 156ms/step - loss: 3.2340 - accuracy: 0.4334


In [160]:
# Predict on test data
predictions = model.predict([X_test, y_test])



13/13 [==============================] - 2s 76ms/step


In [117]:
# Check the shapes and data types of your data
print('X_train shape:', X_train.shape, type(X_train))
print('X_test shape:', X_test.shape, type(X_test))
print('y_train shape:', y_train.shape, type(y_train))
print('y_test shape:', y_test.shape, type(y_test))

X_train shape: (1600, 4) <class 'numpy.ndarray'>
X_test shape: (400, 4) <class 'numpy.ndarray'>
y_train shape: (1600, 4) <class 'numpy.ndarray'>
y_test shape: (400, 4) <class 'numpy.ndarray'>
